# HYSPLIT DATA PREPERATION

Greg Lee
<br>04.27.20

Summary: This notebook represents augmentations used on data in order to prepare it for HYSPLIT

**This is a rough approximation based upon quick google searches. No standardization performed (different websites used). 

** All other information comes from PUBCHEM

Slightly Soluble = 0.1<X<1.0 M
Insoluble = X<=0.1

In [2]:
#USE geo env!

#Basic py: 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import os
import datetime

#Vis
#import seaborn as sns

#Geo
import geopandas as gpd
import fiona
from shapely.geometry import Point
import descartes
import contextily as ctx #Basemaps 

In [246]:
#load the data (please see TRI_Data_Exploration for more information)
tri_df = pd.read_csv('TRI_Carc_FSRel_9099.csv').drop(columns ='Unnamed: 0')

In [247]:
tri_df.head(2)

,CAS_No,Agent,Group,YEAR,TRIFD,FRSID,FACILITYNAME,CITY,COUNTY,ST,...,CAS#/COMPOUNDID,METAL,CARCINOGEN,UNITOFMEASURE,51-FUGITIVEAIR,52-STACKAIR,PRIMARYSIC,INDUSTRYSECTORCODE.1,88-ONE-TIMERELEASE,PRODUCTIONWSTE(81-87)
0,98828,Cumene,2B,1990,84103MCLCM474WE,110000469243,TESORO REFINING & MARKETING COMPANY LLC - SLC,SALT LAKE CITY,SALT LAKE,UT,...,98828,NO,YES,Pounds,0.0,64.0,2911,324,NaN,0.0
1,98828,Cumene,2B,1990,84066PNNZLWESTH,110000913218,PENNZOIL PRODUCTS CO ROOSEVELT REFINERY,ROOSEVELT,DUCHESNE,UT,...,98828,NO,YES,Pounds,100.0,100.0,2911,324,NaN,0.0


# Base Case: January 1990 

### Select the Data and Add Particle Information

In [248]:
#First let's load in the features to seperate the particles and gasses
TRI_chem_info = pd.read_csv('TRI_Carc_Half_Lives.csv')

#keep only those entries with complete information
TRI_chems = TRI_chem_info.dropna(subset = ['CAS'])
TRI_chems = TRI_chems.rename(columns = {'Unnamed: 8':'Phase'})

In [249]:
#Merge that data onto the tri dataframe
tri_df = pd.merge(tri_df,TRI_chem_info,left_on ='CAS_No',right_on = 'CAS',how='left')
tri_df = tri_df.drop(columns = ['CAS','Chems','88-ONE-TIMERELEASE'])

In [251]:
#Seperate into the known differential modeling variants:
fug_par_90 = tri_df[(tri_df['YEAR']==1990) & (tri_df['51-FUGITIVEAIR']>0) & (tri_df['Particle']==1)]
fug_gas_90 = tri_df[(tri_df['YEAR']==1990) & (tri_df['51-FUGITIVEAIR']>0) & (tri_df['Gas']==1) ]
stack_par_90 = tri_df[(tri_df['YEAR']==1990) & (tri_df['52-STACKAIR']>0) & (tri_df['Particle']==1)]
stack_gas_90 = tri_df[(tri_df['YEAR']==1990) & (tri_df['52-STACKAIR']>0) & (tri_df['Gas']==1) ]

print('Fugitive Particle Releases shape: ' + str(fug_par_90.shape))
print('Total number of unique chemicals: '+str(fug_par_90['CAS_No'].drop_duplicates().shape))
print('\nFugitive Gas Releases shape: ' + str(fug_gas_90.shape))
print('Total number of unique chemicals: '+str(fug_gas_90['CAS_No'].drop_duplicates().shape))
print('\nStack Particle Releases shape: ' + str(stack_par_90.shape))
print('Total number of unique chemicals: '+str(stack_par_90['CAS_No'].drop_duplicates().shape))
print('\nStack Gas Releases shape: ' + str(stack_gas_90.shape))
print('Total number of unique chemicals: '+str(stack_gas_90['CAS_No'].drop_duplicates().shape))

Fugitive Particle Releases shape: (13, 32)
Total number of unique chemicals: (3,)

Fugitive Gas Releases shape: (60, 32)
Total number of unique chemicals: (15,)

Stack Particle Releases shape: (12, 32)
Total number of unique chemicals: (5,)

Stack Gas Releases shape: (56, 32)
Total number of unique chemicals: (11,)


### Create a Calender of releases

Information for HYSPLIT
1. Lat/Lon
2. Convert to a datetime object (will need a value every three hours)

In [182]:
#Per each date, we will need to calculate 
cal = pd.date_range("1/1/90 00:00", "12/31/90 21:00", freq="3H")

#Check the shape of the frame
cal.shape[0] == 365*24/3

True

In [183]:
fug_par_90.columns
fug_par_90['UNITOFMEASURE'].drop_duplicates()

26    Pounds
Name: UNITOFMEASURE, dtype: object

In [185]:
final_df = pd.DataFrame(columns = ['Daily_Release','Chem','CAS','MW','Half_Life','Solubility','Lat','Long'])

for rows in range(fug_par_90.shape[0]):
    temp_df = pd.DataFrame({'Daily_Release': [fug_par_90['51-FUGITIVEAIR'].iloc[rows]/cal.shape[0]],
                            'Chem':fug_par_90['CHEMICAL'].iloc[rows],
                            'CAS':fug_par_90['CAS#/COMPOUNDID'].iloc[rows],
                            'MW':fug_par_90['MW'].iloc[rows],
                            'Half_Life':fug_par_90['1/2 Life'].iloc[rows],
                            'Solubility':fug_par_90['Solubility in H2O'].iloc[rows],
                            'Lat':fug_par_90['LATITUDE'].iloc[rows],
                            'Long':fug_par_90['LONGITUDE'].iloc[rows]})

    final_df = final_df.append(temp_df)

In [186]:
final = pd.DataFrame(columns = ['Daily_Release','Chem','CAS','MW','Half_Life','Solubility','Lat','Long'] )
for index in range(test.shape[0]):
    final_df['Date_Time'] = test[index].strftime("%m/%d/%Y %H:%M:%S")
    temp = final_df.set_index('Date_Time')
    final = final.append(temp)

In [187]:
idx = pd.MultiIndex.from_arrays([final.index.array,final['CAS'].array],names = ('Date','CAS'))
final = final.set_index(idx)
final = final.drop(columns = ['CAS'])

In [189]:
final.head()

Daily_Release         Chem      MW Half_Life  \
Date                CAS                                                   
01/01/1990 00:00:00 91203      11.491438  NAPHTHALENE  128.17   8 hours   
                    91203       3.424658  NAPHTHALENE  128.17   8 hours   
                    91203       0.127740  NAPHTHALENE  128.17   8 hours   
                    91203       0.018151  NAPHTHALENE  128.17   8 hours   
                    91203       0.001712  NAPHTHALENE  128.17   8 hours   

                           Solubility        Lat        Long  
Date                CAS                                       
01/01/1990 00:00:00 91203  Insoluble   40.297058 -111.734076  
                    91203  Insoluble   40.824900 -111.923800  
                    91203  Insoluble   40.788900 -111.904410  
                    91203  Insoluble   40.388555 -110.124910  
                    91203  Insoluble   40.886022 -111.904759

#### Let's build a function to do this: 

In [252]:
def chem_date_comb(startdate,enddate,freq,data):
    #Set the time dates for analysis: 
    calendar = pd.date_range(startdate,enddate, freq=freq)
    
    #Collect the daily information from the original data
    final_df = pd.DataFrame(columns = ['Daily_Release','Chem','CAS','MW','Half_Life','Solubility','Lat','Long'])
    for rows in range(data.shape[0]):
        
        #Edit if you want more information added to the release
        temp_df = pd.DataFrame({'Daily_Release': [data['51-FUGITIVEAIR'].iloc[rows]/cal.shape[0]],
                                'Chem':data['CHEMICAL'].iloc[rows],
                                'CAS':data['CAS#/COMPOUNDID'].iloc[rows],
                                'MW':data['MW'].iloc[rows],
                                'Half_Life':data['1/2 Life'].iloc[rows],
                                'Solubility':data['Solubility in H2O'].iloc[rows],
                                'Lat':data['LATITUDE'].iloc[rows],
                                'Long':data['LONGITUDE'].iloc[rows]})

        final_df = final_df.append(temp_df)
    
    #Add a date-time to each  
    final = pd.DataFrame(columns = ['Daily_Release','Chem','CAS','MW','Half_Life','Solubility','Lat','Long'] )
    for index in range(calendar.shape[0]):
        final_df['Date_Time'] = calendar[index].strftime("%m/%d/%Y %H:%M:%S")
        temp = final_df.set_index('Date_Time')
        final = final.append(temp)
    
    #Realign the axis:
    idx = pd.MultiIndex.from_arrays([final.index.array,final['CAS'].array],names = ('Date','CAS'))
    final = final.set_index(idx)
    
    return final.drop(columns = ['CAS'])

In [253]:
#Let's
fug_par_90_df = chem_date_comb(startdate = "1/1/90 00:00",
               enddate = "12/31/90 21:00",
               freq = "3H",
              data = fug_par_90)

fug_gas_90_df = chem_date_comb(startdate = "1/1/90 00:00",
               enddate = "12/31/90 21:00",
               freq = "3H",
              data = fug_gas_90)

stack_par_90_df = chem_date_comb(startdate = "1/1/90 00:00",
               enddate = "12/31/90 21:00",
               freq = "3H",
              data = stack_par_90)

stack_gas_90_df = chem_date_comb(startdate = "1/1/90 00:00",
               enddate = "12/31/90 21:00",
               freq = "3H",
              data = stack_gas_90)

In [254]:
print(fug_par_90_df.shape)
print(fug_gas_90_df.shape)
print(stack_par_90_df.shape)
print(stack_gas_90_df.shape)

(37960, 7)
(175200, 7)
(35040, 7)
(163520, 7)


In [256]:
#Alright these shapes look good!
#Ready for use in HYSPLIT